# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,8.84,84,64,0.45,US,1707613827
1,1,avarua,-21.2078,-159.7750,29.03,70,75,6.17,CK,1707614031
2,2,trujillo,-8.1160,-79.0300,23.98,83,40,5.14,PE,1707613872
3,3,polyarnyy,69.1989,33.4478,-11.07,92,100,2.21,RU,1707614031
4,4,saguenay,48.4168,-71.0657,1.16,94,98,2.67,CA,1707613968


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Ideal cities with: A max temperature lower than 27 degrees but higher than 21, Wind speed less than 4.5 m/s, 
# and Zero cloudiness
ideal_cond_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>21) &(city_data_df["Max Temp"]<27),:]
ideal_cond_cities_df = ideal_cond_cities_df.loc[(ideal_cond_cities_df["Cloudiness"]==0) & (ideal_cond_cities_df["Wind Speed"]<4.5),:]

# Drop any rows with null values
ideal_cond_cities_df_clean = ideal_cond_cities_df.dropna()


# Display sample data
ideal_cond_cities_df_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,90,gunjur,13.2019,-16.7339,23.04,60,0,0.00,GM,1707614040
92,92,kerikeri,-35.2268,173.9474,26.62,48,0,0.89,NZ,1707614040
162,162,harper,4.3750,-7.7169,26.64,86,0,2.89,LR,1707614049
175,175,maraba,-5.3686,-49.1178,25.10,100,0,0.00,BR,1707614051
182,182,les avirons,-21.2418,55.3394,24.03,73,0,3.09,RE,1707614051
259,259,berbera,10.4396,45.0143,25.23,82,0,2.69,SO,1707614059
287,287,odienne,9.5100,-7.5692,24.84,15,0,0.91,CI,1707614062
360,360,timbuktu,16.7735,-3.0074,22.02,8,0,2.74,ML,1707614070
461,461,dara,15.3484,-15.4799,23.99,8,0,4.00,SN,1707614081
470,470,san ignacio,17.1561,-89.0714,25.08,78,0,0.00,BZ,1707614082


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_cities_df_clean[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
90,gunjur,GM,13.2019,-16.7339,60,
92,kerikeri,NZ,-35.2268,173.9474,48,
162,harper,LR,4.3750,-7.7169,86,
175,maraba,BR,-5.3686,-49.1178,100,
182,les avirons,RE,-21.2418,55.3394,73,
259,berbera,SO,10.4396,45.0143,82,
287,odienne,CI,9.5100,-7.5692,15,
360,timbuktu,ML,16.7735,-3.0074,8,
461,dara,SN,15.3484,-15.4799,8,
470,san ignacio,BZ,17.1561,-89.0714,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set the parameters for the type of place
categories = "accommodation.hotel"

# Set parameters to search for a hotel
radius = 10000
limit = 20

# Set up a parameters dictionary
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gunjur - nearest hotel: Footsteps Eco Lodge
kerikeri - nearest hotel: Avalon Resort
harper - nearest hotel: No hotel found
maraba - nearest hotel: Portobelo
les avirons - nearest hotel: Floralys
berbera - nearest hotel: Al Madina Hotel
odienne - nearest hotel: Hotel Frontière
timbuktu - nearest hotel: Hotel La Maison
dara - nearest hotel: No hotel found
san ignacio - nearest hotel: Rainforest Haven Inn
benque viejo el carmen - nearest hotel: Maxim's Palace
goz beida - nearest hotel: No hotel found
kati - nearest hotel: No hotel found
jindabyne - nearest hotel: Lake Jindabyne Hotel
vallenar - nearest hotel: Hotel del Marqués
atar - nearest hotel: فندق سكليل


,City,Country,Lat,Lng,Humidity,Hotel Name
90,gunjur,GM,13.2019,-16.7339,60,Footsteps Eco Lodge
92,kerikeri,NZ,-35.2268,173.9474,48,Avalon Resort
162,harper,LR,4.3750,-7.7169,86,No hotel found
175,maraba,BR,-5.3686,-49.1178,100,Portobelo
182,les avirons,RE,-21.2418,55.3394,73,Floralys
259,berbera,SO,10.4396,45.0143,82,Al Madina Hotel
287,odienne,CI,9.5100,-7.5692,15,Hotel Frontière
360,timbuktu,ML,16.7735,-3.0074,8,Hotel La Maison
461,dara,SN,15.3484,-15.4799,8,No hotel found
470,san ignacio,BZ,17.1561,-89.0714,78,Rainforest Haven Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = ["Hotel Name","Country"],
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)